# About Dataset

Ridership Data source: https://open.toronto.ca/dataset/bike-share-toronto-ridership-data/

Station Location Data source: https://tor.publicbikesystem.net/ube/gbfs/v1/en/station_information

# Initialization

In [28]:
import glob, requests

import pandas as pd

In [29]:
# Specify the path and pattern for the CSV files
# This will look for all CSV files in the specified directory
csv_files = glob.glob('datasets/2018/*.csv')

# Create a list to hold the DataFrames
dataframes = []

# Loop through the list of file paths
for file in csv_files:
    # Read each CSV file and append the DataFrame to the list
    df = pd.read_csv(file)
    dataframes.append(df)

# Concatenate all DataFrames into a single DataFrame 
trips_df = pd.concat(dataframes, ignore_index=True)

trips_df.head(5)

,trip_id,trip_duration_seconds,from_station_id,trip_start_time,from_station_name,trip_stop_time,to_station_id,to_station_name,user_type
0,2383648,393,7018,1/1/2018 0:47,Bremner Blvd / Rees St,1/1/2018 0:54,7176,Bathurst St / Fort York Blvd,Annual Member
1,2383649,625,7184,1/1/2018 0:52,Ossington Ave / College St,1/1/2018 1:03,7191,Central Tech (Harbord St),Annual Member
2,2383650,233,7235,1/1/2018 0:55,Bay St / College St (West Side) - SMART,1/1/2018 0:59,7021,Bay St / Albert St,Annual Member
3,2383651,1138,7202,1/1/2018 0:57,Queen St W / York St (City Hall),1/1/2018 1:16,7020,Phoebe St / Spadina Ave,Annual Member
4,2383652,703,7004,1/1/2018 1:00,University Ave / Elm St,1/1/2018 1:12,7060,Princess St / Adelaide St E,Annual Member


In [30]:
# Check all null values in , if None we can proceed or deal with it if any
trips_df.isnull().sum()

trip_id                  0
trip_duration_seconds    0
from_station_id          0
trip_start_time          0
from_station_name        0
trip_stop_time           0
to_station_id            0
to_station_name          0
user_type                0
dtype: int64

# Adding locations to trips_df

In [31]:
# URL of the JSON data
url = 'https://tor.publicbikesystem.net/ube/gbfs/v1/en/station_information'

# Fetch the JSON data
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Load JSON data
    data = response.json()
    
    # Convert JSON to DataFrame
    bike_stations_df = pd.json_normalize(data['data']['stations'])
    print("Data successfully fetched")
else:
    print(f"Failed to fetch data: {response.status_code}")

Data successfully fetched


In [32]:
bike_stations_df.head(2)

,station_id,name,physical_configuration,lat,lon,altitude,address,capacity,is_charging_station,rental_methods,groups,obcn,short_name,nearby_distance,_ride_code_support,post_code,is_valet_station,cross_street
0,7000,Fort York Blvd / Capreol Ct,REGULAR,43.639832,-79.395954,NaN,Fort York Blvd / Capreol Ct,47,False,"[KEY, TRANSITCARD, CREDITCARD, PHONE]",[South],647-643-9607,647-643-9607,500.0,True,NaN,NaN,NaN
1,7001,Wellesley Station Green P,ELECTRICBIKESTATION,43.664964,-79.383550,NaN,Yonge / Wellesley,23,True,"[KEY, TRANSITCARD, CREDITCARD, PHONE]","[E-Charging , South]",416-617-9576,416-617-9576,500.0,True,M4Y 1G7,NaN,NaN


In [33]:
bike_stations_df.shape

(863, 18)

Converting ID column to Integer

In [34]:
bike_stations_df['station_id'] = bike_stations_df['station_id'].astype(int)

In [35]:
# Delete the specified columns
columns_to_drop = ['physical_configuration', 'altitude', 'address','capacity', 'is_charging_station', 'rental_methods', 'groups', 'obcn', 'short_name', 'nearby_distance', '_ride_code_support', 'post_code', 'is_valet_station', 'cross_street']
bike_stations_df.drop(columns=columns_to_drop, inplace=True)
bike_stations_df.head(3)

,station_id,name,lat,lon
0,7000,Fort York Blvd / Capreol Ct,43.639832,-79.395954
1,7001,Wellesley Station Green P,43.664964,-79.383550
2,7002,St. George St / Bloor St W,43.667131,-79.399555


In [36]:
# Delete "IDs" in bike_stations_df that are not in trips_df
bike_stations_df = bike_stations_df[bike_stations_df['station_id'].isin(trips_df['from_station_id'])].reset_index(drop=True)

# Display the filtered DataFrame
print(bike_stations_df.shape)

(331, 4)


Renaming bike stations to thier old names in 2018

In [37]:
# Create a mapping of station ID to station name from trips_df
name_mapping = trips_df[['from_station_id', 'from_station_name']].drop_duplicates().set_index('from_station_id')['from_station_name'].to_dict()

# Update the station names in bike_stations_df based on the mapping
bike_stations_df['name'] = bike_stations_df['station_id'].map(name_mapping).combine_first(bike_stations_df['name'])

In [38]:
bike_stations_df.head(3)

,station_id,name,lat,lon
0,7000,Fort York Blvd / Capreol Ct,43.639832,-79.395954
1,7001,Lower Jarvis St / The Esplanade,43.664964,-79.383550
2,7002,St. George St / Bloor St W,43.667131,-79.399555


List of coordinates of stations missing data, I mannualy found it

In [39]:
missing_coordinates = {
    # ID: [Name, LAT, LON]
    7060: ['Princess St / Adelaide St E',                   43.652364279365386, -79.38072119893286],
    7177: ['East Liberty St / Pirandello St',               43.639083,          -79.414222],
    7062: ['University Ave / College St (West)',            43.659944,          -79.406389],
    7251: ['The Royal Ontario Museum (Bloor St Entrance)',  43.668444,          -79.405028],
    7011: ['Portland St & Wellington St W',                 43.642917,          -79.405472],
    7051: ['Wellesley St E / Yonge St (Green P)',           43.6655,            -79.395278],
    7275: ['Queen St W / James St',                         43.652364279365386, -79.38072119893286],
    7067: ['Yonge St / Harbour St',                         43.64256307021582,  -79.37620018149019],
    7004: ['University Ave / Elm St',                       43.656365196668915, -79.38781487252689],
    7255: ['Stewart St / Bathurst St  - SMART',             43.64318006677627,  -79.40220532120897],
    7056: ['Parliament St / Gerrard St',                    43.66190892497103,  -79.36689083621276],
    7263: ['Walton St / Elizabeth St - SMART',              43.65775951515591,  -79.38605778701378],
    7167: ['Queen St E / George St (Moss Park)',            43.65391112056363,  -79.37194228233268],
    7058: ['Huron / Harbord St',                            43.663701672289875, -79.40005460166932],
    7099: ['Cherry St / Mill St',                           43.65131162754315,  -79.35721626639824],
    7072: ['Queen St W / Shaw St',                          43.64466219786213,  -79.41611373593442],
    7254: ['Borden St / Bloor St W - SMART',                43.66572398943307,  -79.40878207642875],
    7293: ['College St / McCaul St',                        43.65911010024858,  -79.39345145632274],
    7136: ['Queen St W / Close Ave',                        43.64070304242879,  -79.4359587134529],
    7179: ['Bloor GO / UP Station (West Toronto Railpath)', 43.656427257520185, -79.45167849957652],
    7382: ['Simcoe St / Adelaide St W',                     43.64860574452701,  -79.3864666429806],
    7372: ['King St W / Portland St',                       43.644389289112645, -79.40065141953072],
    7134: ['Marlborough Ave / Yonge St',                    43.67994219829935,  -79.38995166624743],
    7249: ['Dovercourt Rd / Harrison St (Green P) - SMART', 43.65047864724172,  -79.42425799717319],
    7092: ['Pape Subway Green P',                           43.68021607325339,  -79.34413045292051],
    7328: ['Roxborough St W / Yonge St',                    43.67807392458066,  -79.390372189717],
    7330: ['Lee Ave / Queen St E',                          43.6699549059716,   -79.29785451573214],
    7369: ['Dupont St / Emerson Ave - SMART',               43.6671597202503,   -79.44423150877314],
    7219: ['Wolfpack - 7219',                               43.639445987085494, -79.42360693630165]
}

In [40]:
# Dictionary containing the correct coordinates
correct_coordinates = {
    7322: ["King St W / Brant St", 43.645102433201394, -79.39742774871179],
}

# Iterate over the correct_coordinates dictionary to update bike_stations_df
for station_id, coords in correct_coordinates.items():
    bike_stations_df.loc[bike_stations_df['station_id'] == station_id, ['name', 'lat', 'lon']] = coords

In [41]:
# Create a DataFrame from the stations_coordinates
missing_stations_df = pd.DataFrame.from_dict(missing_coordinates, orient='index', columns=['name', 'lat', 'lon']).reset_index()
missing_stations_df.rename(columns={'index': 'station_id'}, inplace=True)

# Merge the DataFrames
bike_stations_df = pd.concat([bike_stations_df, missing_stations_df], ignore_index=True)

bike_stations_df.head(3)

,station_id,name,lat,lon
0,7000,Fort York Blvd / Capreol Ct,43.639832,-79.395954
1,7001,Lower Jarvis St / The Esplanade,43.664964,-79.383550
2,7002,St. George St / Bloor St W,43.667131,-79.399555


In [42]:
bike_stations_df.shape

(360, 4)

Merge bike station locations to trips data

In [43]:
# Merge lat/lon of start stations
trips_df = trips_df.merge(bike_stations_df[['station_id', 'lat', 'lon']], 
                           left_on='from_station_id', right_on='station_id', how='left')

# Rename the lat/lon columns after merging for clarity
trips_df.rename(columns={'lat': 'from_lat', 'lon': 'from_lon'}, inplace=True)

# Merge lat/lon of end stations
trips_df = trips_df.merge(bike_stations_df[['station_id', 'lat', 'lon']], 
                           left_on='to_station_id', right_on='station_id', how='left')

# Rename the lat/lon columns for end stations
trips_df.rename(columns={'lat': 'to_lat', 'lon': 'to_lon'}, inplace=True)

trips_df.drop(["station_id_x", "station_id_y"], axis=1, inplace=True)

# Select only the first occurrence of each column name if we got any
trips_df = trips_df.loc[:, ~trips_df.columns.duplicated()]

In [44]:
trips_df.head(3)

,trip_id,trip_duration_seconds,from_station_id,trip_start_time,from_station_name,trip_stop_time,to_station_id,to_station_name,user_type,from_lat,from_lon,to_lat,to_lon
0,2383648,393,7018,1/1/2018 0:47,Bremner Blvd / Rees St,1/1/2018 0:54,7176,Bathurst St / Fort York Blvd,Annual Member,43.641529,-79.386741,43.639179,-79.399595
1,2383649,625,7184,1/1/2018 0:52,Ossington Ave / College St,1/1/2018 1:03,7191,Central Tech (Harbord St),Annual Member,43.654491,-79.422634,43.661975,-79.407896
2,2383650,233,7235,1/1/2018 0:55,Bay St / College St (West Side) - SMART,1/1/2018 0:59,7021,Bay St / Albert St,Annual Member,43.660087,-79.385655,43.653264,-79.382458


# Changing data types, adding columns and saving memory.

Convert to Datetime format

In [45]:
def convert_dates(df, start_column, end_column):
    # Define the formats you want to support for input parsing
    date_formats = [
        "%d/%m/%Y %H:%M", "%m/%d/%Y %H:%M",
        "%d/%m/%Y %I:%M %p", "%m/%d/%Y %I:%M %p", "%Y-%m-%d %I:%M:%S %p",
        "%Y-%m-%d %H:%M:%S", "%d/%m/%Y %H:%M:%S",
        "%m/%d/%Y %H:%M", "%m/%d/%y %H:%M", "%m/%d/%y %H:%M:%S"
    ]

    # Function to parse dates with specified formats
    def parse_dates(date_series):
        for fmt in date_formats:
            # Attempt to parse dates with the current format
            parsed_dates = pd.to_datetime(date_series, format=fmt, errors='coerce')
            # If parsing results in non-null values, return them
            if parsed_dates.notnull().all():
                return parsed_dates

        # If none of the formats work, return NaT for unparseable dates
        return pd.to_datetime(date_series, errors='coerce')

    # Convert start and end times using the parsing function
    start_dates = parse_dates(df[start_column])
    end_dates = parse_dates(df[end_column])

    # Log unparseable dates
    if start_dates.isnull().any():
        unparseable_starts = df[start_column][start_dates.isnull()]
        print(f"Could not parse start dates: {unparseable_starts.tolist()}")

    if end_dates.isnull().any():
        unparseable_ends = df[end_column][end_dates.isnull()]
        print(f"Could not parse end dates: {unparseable_ends.tolist()}")

    # Convert parsed dates to the desired standard format: %d/%m/%Y %H:%M:%S
    start_dates = start_dates.dt.strftime("%d/%m/%Y %H:%M:%S")
    end_dates = end_dates.dt.strftime("%d/%m/%Y %H:%M:%S")

    return start_dates, end_dates

# Process the merged trips DataFrame
trips_df['trip_start_time'], trips_df['trip_stop_time'] = convert_dates(trips_df, 'trip_start_time', 'trip_stop_time')

# Convert back to datetime type after formatting, if necessary for further analysis
trips_df['trip_start_time'] = pd.to_datetime(trips_df['trip_start_time'], format="%d/%m/%Y %H:%M:%S", errors='coerce')
trips_df['trip_stop_time']  = pd.to_datetime(trips_df['trip_stop_time'], format="%d/%m/%Y %H:%M:%S", errors='coerce')

# Display the updated DataFrame
trips_df.head(3)

,trip_id,trip_duration_seconds,from_station_id,trip_start_time,from_station_name,trip_stop_time,to_station_id,to_station_name,user_type,from_lat,from_lon,to_lat,to_lon
0,2383648,393,7018,2018-01-01 00:47:00,Bremner Blvd / Rees St,2018-01-01 00:54:00,7176,Bathurst St / Fort York Blvd,Annual Member,43.641529,-79.386741,43.639179,-79.399595
1,2383649,625,7184,2018-01-01 00:52:00,Ossington Ave / College St,2018-01-01 01:03:00,7191,Central Tech (Harbord St),Annual Member,43.654491,-79.422634,43.661975,-79.407896
2,2383650,233,7235,2018-01-01 00:55:00,Bay St / College St (West Side) - SMART,2018-01-01 00:59:00,7021,Bay St / Albert St,Annual Member,43.660087,-79.385655,43.653264,-79.382458


Extract hour, day of the week, and month

In [52]:
trips_df['trip_start_hour']        = trips_df['trip_start_time'].dt.hour
trips_df['trip_start_day_of_week'] = trips_df['trip_start_time'].dt.day_name()    # Day of the week as a string (e.g., Monday)
# trips_df['trip_start_month']       = trips_df['trip_start_time'].dt.month_name()  # Month as a string (e.g., January)

Make Days and Month Categorical variable and set thier order

In [53]:
# Sort days of the week for proper order
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
trips_df['trip_start_day_of_week'] = pd.Categorical(trips_df['trip_start_day_of_week'], categories=days_order, ordered=True)
trips_df = trips_df.sort_values('trip_start_day_of_week')

# # Sort months for proper order
# months_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
# trips_df['trip_start_month'] = pd.Categorical(trips_df['trip_start_month'], categories=months_order, ordered=True)
# trips_df = trips_df.sort_values('trip_start_month')

We will retain all ID columns as integers because integers are more space-efficient than strings, especially when managing millions of records. This approach will help optimize our storage and enhance overall performance.

In [54]:
trips_df['from_station_name'] = trips_df['from_station_name'].astype(str)
trips_df['to_station_name']   = trips_df['to_station_name'].astype(str)
trips_df['user_type']         = trips_df['user_type'].astype(str)

In [55]:
trips_df['is_weekend'] = trips_df['trip_start_day_of_week'].apply(lambda x: 'Yes' if x in ['Saturday', 'Sunday'] else 'No')

def categorize_time(hour):
    if 7 <= hour <= 9:
        return 'Morning Rush'
    elif 17 <= hour <= 19:
        return 'Evening Rush'
    else:
        return 'Off Peak'

trips_df['trip_time_category'] = trips_df['trip_start_hour'].apply(categorize_time)

In [56]:
# Convert trips duration seconds to minutes
trips_df['trip_duration_minutes'] = (trips_df['trip_duration_seconds']/60).round(2)
trips_df.drop('trip_duration_seconds', axis=1, inplace=True)

In [57]:
trips_df.sort_values("trip_start_time", ascending=False).head(3)

,trip_id,from_station_id,trip_start_time,from_station_name,trip_stop_time,to_station_id,to_station_name,user_type,from_lat,from_lon,to_lat,to_lon,trip_start_day_of_week,is_weekend,trip_start_hour,trip_time_category,trip_duration_minutes
1933853,4581277,7299,2018-12-31 23:58:00,Mill St / Parliament St,2019-01-01 00:04:00,7013,Scott St / The Esplanade,Annual Member,43.650256,-79.361630,43.646597,-79.375309,Monday,No,23,Off Peak,5.55
1933852,4581276,7014,2018-12-31 23:52:00,Sherbourne St / Carlton St (Allan Gardens),2019-01-01 00:17:00,7269,Toronto Eaton Centre (Yonge St),Annual Member,43.663102,-79.373181,43.655431,-79.380653,Monday,No,23,Off Peak,24.43
1933851,4581275,7020,2018-12-31 23:49:00,Phoebe St / Spadina Ave,2018-12-31 23:55:00,7000,Fort York Blvd / Capreol Ct,Annual Member,43.650033,-79.396555,43.639832,-79.395954,Monday,No,23,Off Peak,5.67


In [60]:
trips_df.columns = ['Trip ID', 'From Station ID', 'Start Time', 'From Station Name', 'Stop Time', 'To Station ID', 'To Station Name', 'Membership Type', 'From Latitude', 'From Longitude', 'To Latitude', 'To Longitude', 'Day of Week', 'Is Weekend', 'Start Hour', 'Time Category', 'Duration (min)']
trips_df.head(3)

,Trip ID,From Station ID,Start Time,From Station Name,Stop Time,To Station ID,To Station Name,Membership Type,From Latitude,From Longitude,To Latitude,To Longitude,Day of Week,Is Weekend,Start Hour,Time Category,Duration (min)
0,2383648,7018,2018-01-01 00:47:00,Bremner Blvd / Rees St,2018-01-01 00:54:00,7176,Bathurst St / Fort York Blvd,Annual Member,43.641529,-79.386741,43.639179,-79.399595,Monday,No,0,Off Peak,6.55
1210115,3761459,7055,2018-08-20 16:21:00,Jarvis St / Carlton St,2018-08-20 16:39:00,7106,Mill St / Tannery Rd,Casual Member,43.662070,-79.376170,43.652219,-79.354455,Monday,No,16,Off Peak,17.72
1210114,3761458,7264,2018-08-20 16:21:00,Bloor St E / Huntley St - SMART,2018-08-20 16:29:00,7028,Gould St / Mutual St,Annual Member,43.671535,-79.379173,43.658200,-79.376800,Monday,No,16,Off Peak,7.93


In [61]:
trips_df.to_csv('datasets/2018/bikeshare_toronto_dashboard.csv', index=False)